In [1]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import logging
from flask import Flask, request, jsonify

# Set up logging
logging.basicConfig(level=logging.INFO)

# Initialize Flask app
app = Flask(__name__)

class ArXivSearch:
    BASE_URL = 'http://export.arxiv.org/api/query?'

    @staticmethod
    def search_arxiv(keywords, num_papers, start_date, end_date):
        """
        Search arXiv for papers based on keywords and date range.
        
        :param keywords: Search keywords
        :param num_papers: Number of papers to retrieve
        :param start_date: Start date for the search (datetime.date object)
        :param end_date: End date for the search (datetime.date object)
        :return: List of dictionaries containing paper information
        """
        query = f'search_query=all:{keywords}&start=0&max_results={num_papers}'
        query += f'&submittedDate:[{start_date.strftime("%Y-%m-%d")} TO {end_date.strftime("%Y-%m-%d")}]'
        
        url = ArXivSearch.BASE_URL + query

        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for bad status codes
        except requests.RequestException as e:
            logging.error(f"Failed to retrieve data: {str(e)}")
            return []

        root = ET.fromstring(response.content)
        namespace = {'atom': 'http://www.w3.org/2005/Atom'}
        entries = root.findall('atom:entry', namespace)

        results = []
        for entry in entries:
            title = entry.find('atom:title', namespace).text.strip()
            authors = entry.findall('atom:author/atom:name', namespace)
            author_names = [author.text for author in authors]
            published = entry.find('atom:published', namespace).text
            id_url = entry.find('atom:id', namespace).text
            pdf_url = entry.find('atom:link[@title="pdf"]', namespace).get('href')
            arxiv_id = id_url.split('/abs/')[-1]
            
            results.append({
                "title": title,
                "authors": ', '.join(author_names),
                "published": published,
                "arxiv_url": id_url,
                "pdf_url": pdf_url,
                "arxiv_id": arxiv_id
            })

        return results

    @staticmethod
    def get_pdf_content(url):
        """
        Retrieve PDF content from a given URL.
        
        :param url: URL of the PDF
        :return: Content of the PDF as bytes
        """
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.content
        except requests.RequestException as e:
            logging.error(f"Failed to retrieve PDF: {str(e)}")
            return None

@app.route('/search_arxiv', methods=['GET'])
def search_arxiv():
    keywords = request.args.get('keywords', default='machine learning', type=str)
    num_papers = request.args.get('num_papers', default=5, type=int)
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=30)  # Last 30 days

    results = ArXivSearch.search_arxiv(keywords, num_papers, start_date, end_date)
    return jsonify(results)

if __name__ == "__main__":
    app.run(debug=True)

 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, 

SystemExit: 1

/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [2]:
from PyPaperBot import PyPaperBot

bot = PyPaperBot(
    query="Machine learning",
    scholar_pages=range(3),
    min_date=2018,
    scholar_results=10,
    restrict=True
)
papers = bot()


Google Scholar page 0 : 10 papers found
Searching paper 1 of 6 on Crossref...
Searching paper 2 of 6 on Crossref...
Searching paper 3 of 6 on Crossref...
Searching paper 4 of 6 on Crossref...
Searching paper 5 of 6 on Crossref...
Searching paper 6 of 6 on Crossref...
Papers found on Crossref: 6/6


Google Scholar page 1 : 10 papers found
Searching paper 1 of 6 on Crossref...
Searching paper 2 of 6 on Crossref...


KeyboardInterrupt: 

In [ ]:
from PyPaperBot import PyPaperBot

bot = PyPaperBot(
    query="Machine learning",
    scholar_pages=range(3),
    min_date=2018,
    scholar_results=10,
    restrict=True
)
papers = bot()

In [5]:
import PyPaperBot as bot

# Create a PyPaperBot instance
paperbot = bot.PyPaperBot(
    query="Machine learning",
    scholar_pages=range(1),
    min_date=2018,
    scholar_results=10,
    restrict=True
)

# Get papers information
papers = paperbot()

# Print paper details
for i, paper in enumerate(papers, 1):
    print(f"\nPaper {i}:")
    print(f"Title: {paper.title}")
    print(f"Authors: {paper.authors}")
    print(f"Year: {paper.year}")
    print(f"DOI: {paper.DOI}")
    print("-" * 50)

# Download papers if needed
download_dir = r"C:\Users\Octane\Desktop\project-bolt-sb1-mjsjn-main\project\projects\nhj\downloaded"
bot.PyPaperBot.downloadPapers(
    papers=papers,
    dwnl_dir=download_dir,
    num_limit=None,
    SciHub_URL="https://sci-hub.do"
)


Google Scholar page 0 : 10 papers found
Searching paper 1 of 6 on Crossref...
Searching paper 2 of 6 on Crossref...
Searching paper 3 of 6 on Crossref...
Searching paper 4 of 6 on Crossref...
Searching paper 5 of 6 on Crossref...
Searching paper 6 of 6 on Crossref...
Papers found on Crossref: 6/6


Paper 1:
Title: Machine learning algorithms-a review
Authors: B Mahesh
Year: 2020
DOI: 10.21275/sr22815163219
--------------------------------------------------

Paper 2:
Title: Machine learning and law: An overview
Authors: H Surden
Year: 2021
DOI: 10.4337/9781788972826.00014
--------------------------------------------------

Paper 3:
Title: Machine learning and the physical sciences
Authors: None
Year: 2019
DOI: 10.1201/9781003350101
--------------------------------------------------

Paper 4:
Title: What is machine learning? A primer for the epidemiologist
Authors: None
Year: 2019
DOI: 10.1093/aje/kwz189
--------------------------------------------------

Paper 5:
Title: International c

KeyboardInterrupt: 

In [2]:
import requests
import xml.etree.ElementTree as ET
from datetime import datetime, timedelta
import logging
from flask import Flask, request, jsonify
import urllib.parse

# Set up logging
logging.basicConfig(level=logging.INFO)

# Initialize Flask app
app = Flask(__name__)

class ArXivSearch:
    BASE_URL = 'http://export.arxiv.org/api/query?'

    @staticmethod
    def search_arxiv(keywords, num_papers, start_date, end_date):
        """
        Search arXiv for papers based on keywords and date range.

        :param keywords: Search keywords
        :param num_papers: Number of papers to retrieve
        :param start_date: Start date for the search (datetime.date object)
        :param end_date: End date for the search (datetime.date object)
        :return: List of dictionaries containing paper information
        """
        # URL-encode keywords
        encoded_keywords = urllib.parse.quote(keywords)
        query = f'search_query=all:{encoded_keywords}&start=0&max_results={num_papers}'
        query += f'&submittedDate:[{start_date.strftime("%Y-%m-%d")} TO {end_date.strftime("%Y-%m-%d")}]'
        
        url = ArXivSearch.BASE_URL + query
        logging.info(f"Querying arXiv API with URL: {url}")

        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise exception for bad HTTP status codes
        except requests.RequestException as e:
            logging.error(f"Failed to retrieve data: {str(e)}")
            return []

        # Parse the XML response
        try:
            root = ET.fromstring(response.content)
        except ET.ParseError as e:
            logging.error(f"Failed to parse XML response: {str(e)}")
            return []

        namespace = {'atom': 'http://www.w3.org/2005/Atom'}
        entries = root.findall('atom:entry', namespace)

        results = []
        for entry in entries:
            # Extract details with error handling
            title = entry.find('atom:title', namespace).text.strip() if entry.find('atom:title', namespace) else "Unknown"
            authors = entry.findall('atom:author/atom:name', namespace)
            author_names = [author.text for author in authors if author is not None]
            published = entry.find('atom:published', namespace).text if entry.find('atom:published', namespace) else "Unknown"
            id_url = entry.find('atom:id', namespace).text if entry.find('atom:id', namespace) else "Unknown"

            # Find the PDF URL
            pdf_url = None
            links = entry.findall('atom:link', namespace)
            for link in links:
                if link.attrib.get('type') == 'application/pdf':
                    pdf_url = link.attrib.get('href')
                    break

            arxiv_id = id_url.split('/abs/')[-1] if '/abs/' in id_url else "Unknown"
            
            results.append({
                "title": title,
                "authors": ', '.join(author_names),
                "published": published,
                "arxiv_url": id_url,
                "pdf_url": pdf_url,
                "arxiv_id": arxiv_id
            })

        return results

    @staticmethod
    def get_pdf_content(url):
        """
        Retrieve PDF content from a given URL.

        :param url: URL of the PDF
        :return: Content of the PDF as bytes
        """
        try:
            response = requests.get(url)
            response.raise_for_status()
            return response.content
        except requests.RequestException as e:
            logging.error(f"Failed to retrieve PDF: {str(e)}")
            return None

@app.route('/search_arxiv', methods=['GET'])
def search_arxiv():
    keywords = request.args.get('keywords', default='machine learning', type=str)
    num_papers = request.args.get('num_papers', default=5, type=int)
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=30)  # Last 30 days

    results = ArXivSearch.search_arxiv(keywords, num_papers, start_date, end_date)
    return jsonify(results)

if __name__ == "__main__":
    app.run(debug=True, port=5001)



 * Serving Flask app '__main__'
 * Debug mode: on


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5001
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug: * Restarting with watchdog (inotify)
Traceback (most recent call last):
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/traitlets/config/application.py", line 1074, in launch_instance
    app.initialize(argv)
  File "/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/traitlets/config/application.py", line 118, in inner
    return method(app, 

SystemExit: 1

/home/zack/anaconda3/envs/miner/lib/python3.10/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [1]:
!pip install paperswithcode-client

  Using cached paperswithcode_client-0.3.1-py3-none-any.whl.metadata (7.2 kB)
  Using cached tea_client-0.0.7-py3-none-any.whl.metadata (750 bytes)
  Using cached tea_console-0.0.6-py3-none-any.whl.metadata (816 bytes)
  Using cached typer-0.3.2-py3-none-any.whl.metadata (12 kB)
  Using cached httpx-0.14.3-py3-none-any.whl.metadata (33 kB)
  Using cached pydantic-1.6.2-py36.py37.py38-none-any.whl.metadata (73 kB)
  Using cached tea-0.1.7-py3-none-any.whl.metadata (1.6 kB)
  Using cached pytz-2021.3-py2.py3-none-any.whl.metadata (21 kB)
  Using cached rich-9.11.1-py3-none-any.whl.metadata (16 kB)
  Using cached tzlocal-2.1-py2.py3-none-any.whl.metadata (8.2 kB)
  Using cached click-7.1.2-py2.py3-none-any.whl.metadata (2.9 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl.metadata (3.2 kB)
  Using cached rfc3986-1.5.0-py2.py3-none-any.whl.metadata (6.5 kB)
  Using cached httpcore-0.10.2-py3-none-any.whl.metadata (6.3 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)

  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [36 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-311\psutil
      copying psutil\_common.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_compat.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_psaix.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_psbsd.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_pslinux.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_psosx.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_psposix.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_pssunos.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\_pswindows.py -> build\lib.win-amd64-cpython-311\psutil
      copying psutil\__init__.py -> build\lib.win-amd64-cpython-3

: 

In [6]:
import xmltodict
import requests

class ScopusSearcher:
    @staticmethod
    def search_scopus(keywords, num_papers, start_date, end_date):
        """
        Search Scopus database for papers based on keywords and date range.

        :param keywords: Search keywords
        :param num_papers: Number of papers to retrieve
        :param start_date: Start date for the search (datetime.date object)
        :param end_date: End date for the search (datetime.date object)
        :return: List of dictionaries containing paper information
        """
        query = f"TITLE-ABS-KEY({keywords}) AND PUBYEAR IS {start_date.year} AND PUBYEAR IS {end_date.year}"
        url = f"http://api.elsevier.com/content/search/scopus?query={query}&httpAccept=application%2Fxml&apiKey=3c2de5e6e3b0e1f2d5a6ae1f2f5b6a7c8&start=0&count={num_papers}"
        response = requests.get(url)
        response.raise_for_status()
        data = xmltodict.parse(response.content)
        results = []
        for entry in data['search-results']['entry']:
            arxiv_id = entry['prism:doi'] if 'prism:doi' in entry else None
            title = entry['dc:title'] if 'dc:title' in entry else None
            authors = entry['author'] if 'author' in entry else []
            author_names = [author['$t'] for author in authors if 'author' in author]
            published = entry['prism:coverDate'] if 'prism:coverDate' in entry else None
            results.append({
                "title": title,
                "authors": ', '.join(author_names),
                "published": published,
                "arxiv_url": None,
                "pdf_url": None,
                "arxiv_id": arxiv_id
            })
        return results

if __name__ == "__main__":
    api_key = "7f59af901d2d86f78a1fd60c1bf9426a"
    keywords = "llm"
    num_papers = 10
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=30)  # Last 30 days
    results = ScopusSearcher.search_scopus(keywords, num_papers, start_date, end_date)
    print(json.dumps(results, indent=4))



AttributeError: module 'datetime' has no attribute 'now'

In [7]:
import xmltodict
import requests
import json
from datetime import datetime, timedelta

class ScopusSearcher:
    @staticmethod
    def search_scopus(keywords, num_papers, start_date, end_date):
        """
        Search Scopus database for papers based on keywords and date range.

        :param keywords: Search keywords
        :param num_papers: Number of papers to retrieve
        :param start_date: Start date for the search (datetime.date object)
        :param end_date: End date for the search (datetime.date object)
        :return: List of dictionaries containing paper information
        """
        query = f"TITLE-ABS-KEY({keywords}) AND PUBYEAR IS {start_date.year} AND PUBYEAR IS {end_date.year}"
        url = f"http://api.elsevier.com/content/search/scopus?query={query}&httpAccept=application%2Fxml&apiKey=3c2de5e6e3b0e1f2d5a6ae1f2f5b6a7c8&start=0&count={num_papers}"
        response = requests.get(url)
        response.raise_for_status()
        data = xmltodict.parse(response.content)
        results = []
        for entry in data['search-results']['entry']:
            arxiv_id = entry['prism:doi'] if 'prism:doi' in entry else None
            title = entry['dc:title'] if 'dc:title' in entry else None
            authors = entry['author'] if 'author' in entry else []
            author_names = [author['$t'] for author in authors if 'author' in author]
            published = entry['prism:coverDate'] if 'prism:coverDate' in entry else None
            results.append({
                "title": title,
                "authors": ', '.join(author_names),
                "published": published,
                "arxiv_url": None,
                "pdf_url": None,
                "arxiv_id": arxiv_id
            })
        return results

if __name__ == "__main__":
    api_key = "7f59af901d2d86f78a1fd60c1bf9426a"  # Consider replacing this with a secure method
    keywords = "llm"
    num_papers = 10
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=30)  # Last 30 days
    results = ScopusSearcher.search_scopus(keywords, num_papers, start_date, end_date)
    print(json.dumps(results, indent=4))


HTTPError: 401 Client Error: Unauthorized for url: https://api.elsevier.com/content/search/scopus?query=TITLE-ABS-KEY(llm)%20AND%20PUBYEAR%20IS%202024%20AND%20PUBYEAR%20IS%202024&httpAccept=application%2Fxml&apiKey=3c2de5e6e3b0e1f2d5a6ae1f2f5b6a7c8&start=0&count=10

In [10]:
import pybliometrics.scopus as scopus


AttributeError: module 'pybliometrics.scopus' has no attribute 'SearchScopus'

In [8]:
pip install pybliometrics

Note: you may need to restart the kernel to use updated packages.


In [13]:
import xmltodict
import requests
import json
from datetime import datetime, timedelta

class ScopusSearcher:
    @staticmethod
    def search_scopus(keywords, num_papers, start_date, end_date):
        """
        Search Scopus database for papers based on keywords and date range.

        :param keywords: Search keywords
        :param num_papers: Number of papers to retrieve
        :param start_date: Start date for the search (datetime.date object)
        :param end_date: End date for the search (datetime.date object)
        :return: List of dictionaries containing paper information
        """
        query = f"TITLE-ABS-KEY({keywords}) AND PUBYEAR IS {start_date.year} AND PUBYEAR IS {end_date.year}"
        
        # URL for the Scopus API
        url = "https://api.elsevier.com/content/search/scopus"
        
        # API Key - Include it in the Authorization header
        headers = {
            'Authorization': '9d9599622503af5c26c4718b85ffbc61',  # Your API key
            'Accept': 'application/xml',  # Accept XML response
        }
        
        # Query parameters
        params = {
            'query': query,
            'start': 0,
            'count': num_papers  # Number of results to retrieve
        }
        
        # Sending the GET request
        response = requests.get(url, headers=headers, params=params)
        
        # Raise error if response status is not OK (200)
        response.raise_for_status()
        
        # Parse the XML response
        data = xmltodict.parse(response.content)
        
        results = []
        for entry in data['search-results']['entry']:
            arxiv_id = entry['prism:doi'] if 'prism:doi' in entry else None
            title = entry['dc:title'] if 'dc:title' in entry else None
            authors = entry['author'] if 'author' in entry else []
            author_names = [author['$t'] for author in authors if 'author' in author]
            published = entry['prism:coverDate'] if 'prism:coverDate' in entry else None
            
            results.append({
                "title": title,
                "authors": ', '.join(author_names),
                "published": published,
                "arxiv_url": None,  # Could be constructed if DOI is available
                "pdf_url": None,  # If available
                "arxiv_id": arxiv_id
            })
        return results

if __name__ == "__main__":
    keywords = "llm"  # Example search term
    num_papers = 10  # Number of results to retrieve
    end_date = datetime.now().date()
    start_date = end_date - timedelta(days=30)  # Last 30 days
    
    # Search for papers
    results = ScopusSearcher.search_scopus(keywords, num_papers, start_date, end_date)
    
    # Print the results
    print(json.dumps(results, indent=4))


HTTPError: 401 Client Error: Unauthorized for url: https://api.elsevier.com/content/search/scopus?query=TITLE-ABS-KEY%28llm%29+AND+PUBYEAR+IS+2024+AND+PUBYEAR+IS+2024&start=0&count=10

In [14]:
import requests
import json

# Define the API endpoint and key
url = "https://api.elsevier.com/content/search/scopus"
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Parameters for the query
params = {
    "query": "all(gene)",  # The search query
    "apiKey": api_key,     # The API key
    "httpAccept": "application/json",  # Request JSON format
    "count": 10            # Number of results
}

# Send the GET request
response = requests.get(url, params=params)

# Check the response status
if response.status_code == 200:
    # Parse and pretty-print the JSON response
    data = response.json()
    print(json.dumps(data, indent=4))
else:
    # Print an error message
    print(f"Failed to retrieve data. HTTP Status: {response.status_code}")
    print(response.text)


{
    "search-results": {
        "opensearch:totalResults": "9466696",
        "opensearch:startIndex": "0",
        "opensearch:itemsPerPage": "10",
        "opensearch:Query": {
            "@role": "request",
            "@searchTerms": "all(gene)",
            "@startPage": "0"
        },
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=10&query=all%28gene%29&apiKey=7f59af901d2d86f78a1fd60c1bf9426a&httpAccept=application%2Fjson",
                "@type": "application/json"
            },
            {
                "@_fa": "true",
                "@ref": "first",
                "@href": "https://api.elsevier.com/content/search/scopus?start=0&count=10&query=all%28gene%29&apiKey=7f59af901d2d86f78a1fd60c1bf9426a&httpAccept=application%2Fjson",
                "@type": "application/json"
            },
            {
                "@_fa": "true",
  

In [1]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210601445",
        "dc:

: 

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210601445",
        "dc:

: 

In [1]:
import json

# Define a mock function to simulate the API request since outgoing connections are restricted in this environment.
def mock_api_request(endpoint, headers, data):
    print("Mock API request initiated.")
    print(f"Endpoint: {endpoint}")
    print(f"Headers: {headers}")
    print(f"Data: {data}")
    # Simulate a successful response
    return {"status": "success", "message": "Synchronization successful."}

# Define the API endpoint and token
API_ENDPOINT = "https://paperswithcode.com/api/v1/evaluation/synchronize/"
API_TOKEN = "a127638a859370f39ed282d14f6dd9456d073c54"

# Define the evaluation table sync request data
evaluation_request = {
    "task": "Image Classification",
    "dataset": "VeryTinyImageNet",
    "description": "Optional description of your challenge in markdown format",
    "metrics": [
        {"name": "Top 1 Accuracy", "is_loss": False},
        {"name": "Top 5 Accuracy", "is_loss": False}
    ],
    "results": [
        {
            "metrics": {
                "Top 1 Accuracy": 85,
                "Top 5 Accuracy": 95
            },
            "paper": "",
            "methodology": "My Unpublished Model Name",
            "external_id": "competition-submission-id-4321",
            "evaluated_on": "2020-11-20",
            "external_source_url": "https://my.competition.com/leaderboard/entry1"
        },
        {
            "metrics": {
                "Top 1 Accuracy": 75,
                "Top 5 Accuracy": 81
            },
            "paper": "https://arxiv.org/abs/1512.03385",
            "methodology": "ResNet-50 (baseline)",
            "external_id": "competition-submission-id-1123",
            "evaluated_on": "2020-09-20",
            "external_source_url": "https://my.competition.com/leaderboard/entry2"
        }
    ]
}

# Console-based interface
print("Papers with Code Evaluation Synchronization")
print("This program simulates the synchronization of evaluation results with the Papers with Code API.")

# Proceed without waiting for user input
print("Synchronizing evaluation data...")

try:
    # Mock the API request instead of sending a real one
    headers = {
        "Authorization": f"Token {API_TOKEN}",
        "Content-Type": "application/json"
    }
    response = mock_api_request(API_ENDPOINT, headers, json.dumps(evaluation_request))

    # Display the response
    if response.get("status") == "success":
        print(response["message"])
    else:
        print("Mock API returned an error.")

except Exception as e:
    # Display the error response for debugging purposes
    print("Error occurred during synchronization:")
    print(str(e))


ModuleNotFoundError: No module named 'paperswithcode'

In [3]:

from paperswithcode import PapersWithCodeClient

client = PapersWithCodeClient()
papers = client.paper_list()
print(papers.results[0])
print(papers.next_page)

ModuleNotFoundError: No module named 'paperswithcode'

: 

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210601445",
        "dc:

: 

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210601445",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210601445",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210601445",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210601445?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210601445&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210601445",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [ ]:
from elsapy.elsclient import ElsClient
from elsapy.elssearch import ElsSearch
import json

# Your Scopus API Key
api_key = "7f59af901d2d86f78a1fd60c1bf9426a"

# Initialize the Elsevier API client
client = ElsClient(api_key)

# Define the search query
query = "ALL(gene)"

# Perform the search
try:
    # Create an ElsSearch object
    scopus_search = ElsSearch(query, "scopus")
    
    # Execute the search
    scopus_search.execute(client, get_all=False)  # `get_all=False` limits to the first page of results
    
    # Print results in JSON format
    print(json.dumps(scopus_search.results, indent=4))
except Exception as e:
    print(f"An error occurred: {e}")


[
    {
        "@_fa": "true",
        "link": [
            {
                "@_fa": "true",
                "@ref": "self",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211"
            },
            {
                "@_fa": "true",
                "@ref": "author-affiliation",
                "@href": "https://api.elsevier.com/content/abstract/scopus_id/85210991211?field=author,affiliation"
            },
            {
                "@_fa": "true",
                "@ref": "scopus",
                "@href": "https://www.scopus.com/inward/record.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            },
            {
                "@_fa": "true",
                "@ref": "scopus-citedby",
                "@href": "https://www.scopus.com/inward/citedby.uri?partnerID=HzOxMe3b&scp=85210991211&origin=inward"
            }
        ],
        "prism:url": "https://api.elsevier.com/content/abstract/scopus_id/85210991211",
        "dc:

In [5]:
!lsof -i :9002


COMMAND    PID USER   FD   TYPE  DEVICE SIZE/OFF NODE NAME
windsurf 73075 zack  102u  IPv4 1088577      0t0  TCP localhost:52770->localhost:9002 (ESTABLISHED)
python   73415 zack   10u  IPv4 1087747      0t0  TCP localhost:9002 (LISTEN)
python   73415 zack   17u  IPv4 1089854      0t0  TCP localhost:52742->localhost:9002 (ESTABLISHED)
python   73415 zack   18u  IPv4 1089855      0t0  TCP localhost:9002->localhost:52742 (ESTABLISHED)
python   73415 zack   52u  IPv4 1089859      0t0  TCP localhost:9002->localhost:52770 (ESTABLISHED)


In [5]:
!kill -9 9002


/bin/bash: line 1: kill: (9002) - No such process


In [ ]:
!lsof -i -P -n | grep LISTEN | awk '{print $2}' | xargs kill -9
